# LeNet-5 (1998) – NumPy Implementation from Scratch

In this notebook I build LeNet-5, the CNN introduced by
Yann LeCun for handwritten digit recognition.

* **Dataset** : MNIST (60 k train / 10 k test)  
* **Input size** : LeNet expects 32 × 32 greyscale images.  
  MNIST is 28 × 28, so we pad 2 pixels on each side in the first
  convolution layer.


## Network Architecture

| Layer      | Type                 | Parameters                             | Output Shape (input 28x28) |
| ---------- | -------------------- | -------------------------------------- | -------------------------- |
| **C1**     | Convolution          | 6 filters, 5×5 kernel, stride=1, pad=2 | (6, 28, 28)                |
|            | Activation (Sigmoid) |                                        | (6, 28, 28)                |
| **S2**     | Average Pooling      | 2×2 window, stride=2                   | (6, 14, 14)                |
| **C3**     | Convolution          | 16 filters, 5×5 kernel, stride=1       | (16, 10, 10)               |
|            | Activation (Sigmoid) |                                        | (16, 10, 10)               |
| **S4**     | Average Pooling      | 2×2 window, stride=2                   | (16, 5, 5)                 |
| **C5**     | Convolution          | 120 filters, 5×5 kernel, stride=1      | (120, 1, 1)                |
|            | Activation (Sigmoid) |                                        | (120,)                     |
| **F6**     | Fully Connected      | 120 → 84                               | (84,)                      |
|            | Activation (Sigmoid) |                                        | (84,)                      |
| **Output** | Fully Connected      | 84 → 10                                | (10,)                      |

![Architecture](figures/Architecture.png)

Image source: Zhang, Aston and Lipton, Zachary C. and Li, Mu and Smola, Alexander J. - https://github.com/d2l-ai/d2l-en

## 1  Load and preprocess MNIST
Load the MNIST dataset from sklearn, normalize it to [0,1], and reshape for convolution layers.


In [3]:
import numpy as np
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)

X = mnist['data']       # Shape: (70000, 784)
y = mnist['target']     # Shape: (70000,)

X = X / 255.0           # Normalize pixel values to [0, 1]
y = y.astype(np.int32)  # Convert labels to integers

X = X.reshape(-1, 1, 28, 28) # Reshape for CNN

In [5]:
# Split into train/test (10k train, 10k test)
X_train, X_test = X[:10000], X[10000:20000]
y_train, y_test = y[:10000], y[10000:20000]

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (10000, 1, 28, 28)
X_test shape: (10000, 1, 28, 28)
y_train shape: (10000,)
y_test shape: (10000,)


In [3]:
y_train

array([5, 0, 4, ..., 6, 9, 7], dtype=int32)

## 2  Utility functions
Activation, and loss. Documentation is generated by ChatGPT 4o.

In [6]:
def sigmoid(x):
    """
    Element-wise Sigmoid activation function.

    Parameters:
    - x: np.ndarray or float
        Input value(s).

    Returns:
    - np.ndarray or float
        Output after applying the sigmoid function element-wise.
    """
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    """
    Derivative of the sigmoid function.

    Parameters:
    - x: np.ndarray or float
        Input value(s).

    Returns:
    - np.ndarray or float
        The derivative of the sigmoid function evaluated at x.
    """
    s = sigmoid(x)
    return s * (1 - s)

def tanh(x):
    """
    Element-wise hyperbolic tangent (tanh) activation function.

    Parameters:
    - x: np.ndarray or float
        Input value(s).

    Returns:
    - np.ndarray or float
        Output after applying the tanh function element-wise.
    """
    return np.tanh(x)

def tanh_deriv(x):
    """
    Derivative of the hyperbolic tangent (tanh) function.

    Parameters:
    - x: np.ndarray or float
        Input value(s).

    Returns:
    - np.ndarray or float
        The derivative of tanh, computed as 1 - tanh(x)^2, evaluated at x.
    """
    return 1.0 - np.tanh(x)**2

def softmax(Z: np.ndarray) -> np.ndarray:
    """
    Compute softmax probabilities over classes.

    Parameters:
    - Z: np.ndarray
        Logits of shape (batch_size, num_classes).

    Returns:
    - np.ndarray
        Softmax probabilities of shape (batch_size, num_classes).
    """
    Z = Z - np.max(Z, axis=1, keepdims=True)
    exp_Z = np.exp(Z)
    return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)

def CrossEntropy(yhat: np.ndarray, y: np.ndarray, eps: float = 1e-15) -> float:
    """
    Mean cross-entropy loss for multi-class classification.

    Parameters:
    - yhat: np.ndarray
        Predicted probabilities (batch_size, num_classes).
    - y: np.ndarray
        One-hot encoded true labels (batch_size, num_classes).
    - eps: float, optional
        Small epsilon to avoid log(0).

    Returns:
    - float
        Mean cross-entropy loss over the batch.
    """
    yhat = np.clip(yhat, eps, 1 - eps)
    return -np.mean(np.sum(y * np.log(yhat), axis=1))


## Zero Padding

Pad the input to get the 32×32 images LeNet-5 was designed for.  
This function does zero-padding around the spatial dimensions.


In [7]:
def padding(X: np.ndarray, p: int) -> np.ndarray:
    """
    Pads images with zeros.

    Parameters:
    - X: Input tensor of shape (batch, channels, height, width)
    - p: Padding size

    Returns:
    - Zero-padded tensor of shape (batch, channels, height + 2*p, width + 2*p)
    """
    batch, channel, height, width = X.shape
    Y = np.zeros((batch, channel, height + 2*p, width + 2*p))
    Y[:, :, p:p+height, p:p+width] = X
    return Y

## Convolution Layer (Forward)

This function performs a multi-channel, multi-filter convolution, padding inputs as needed.
We implement the forward pass using for loops, summing over input channels and applying each filter.


In [8]:
def Convolution(X, p, in_channels, out_channels, K, bias):
    """
    Performs a convolution over multi-channel input with multiple output channels.
    
    Arguments:
    - X: input tensor of shape (batch, in_channels, height, width)
    - padding: number of padding pixels on each side
    - in_channels: number of input channels
    - out_channels: number of output channels (number of filters)
    - K: Kernel
    - bias: Bias term added to each output channel
    
    Returns:
    - Y: output tensor of shape (batch, out_channels, output_height, output_width)
    - X: Padded input for backpropagation
    - K: The kernel to be reused
    - b: The bias to be reused
    """

    # Pad input
    if p > 0:
        X = padding(X, p)

    batch, channel, height, width = X.shape

    # Compute output size (assuming stride=1)
    output_height = height - K.shape[2] + 1
    output_width  = width - K.shape[3] + 1
    Y = np.zeros(shape=(batch, out_channels, output_height, output_width))

    """
    First attempt. Worked but was really slow. Kept here to showcase my struggles.

    for b in range(batch):
        for out_ch in range(out_channels):
            for in_ch in range(in_channels):
                for h in range(output_height):
                    for w in range(output_width):
                        Y[b, out_ch, h, w] += np.sum(
                            X[b, in_ch, h: h + K.shape[2], w: w + K.shape[3]] * 
                            K[out_ch, in_ch]
            Y[b, out_ch, :, :] += bias[out_ch]
    """
    # Vectorized height and width loop.-
    for b in range(batch):
        for out_ch in range(out_channels):
            accum = np.zeros((output_height, output_width))
            # Sum over each input channel
            for in_ch in range(in_channels):
                # This uses a 2D sliding window, no inner h,w loop
                for i in range(K.shape[2]):
                    for j in range(K.shape[3]):
                        accum += X[b, in_ch, i:i+output_height, j:j+output_width] * K[out_ch, in_ch, i, j]
            # Add bias
            Y[b, out_ch] = accum + bias[out_ch]

    return Y, X, K, bias

## Convolution Backward Pass

Computes gradients for weights, biases, and inputs.
This is a direct implementation of the chain rule for convolution layers. This was by far the most difficult part of the project and many errors appeared.


In [9]:
def ConvBackward(dY, X, K):
    """
    Backpropagation for a convolution layer.

    Computes gradients w.r.t.:
    - input X
    - kernel weights K
    - bias

    Parameters:
    - dY: Gradient of loss w.r.t. output, shape (batch, out_channels, out_height, out_width)
    - X: Input to the convolution, shape (batch, in_channels, height, width)
    - K: Kernels, shape (out_channels, in_channels, kH, kW)

    Returns:
    - dX: Gradient w.r.t. input X
    - dK: Gradient w.r.t. kernels
    - db: Gradient w.r.t. biases (averaged over batch)
    """
    batch, in_channels, height, width = X.shape
    out_channels, _, kH, kW = K.shape
    out_height, out_width = dY.shape[2], dY.shape[3]

    dK = np.zeros_like(K)
    db = np.sum(dY, axis=(0,2,3)) / batch
    dX = np.zeros_like(X)

    for b in range(batch):
        for out_ch in range(out_channels):
            for in_ch in range(in_channels):
                for i in range(kH):
                    for j in range(kW):
                        h_end = i + out_height
                        w_end = j + out_width
                        # clip if we overshoot the input dims
                        if h_end > height or w_end > width:
                            h_end = min(h_end, height)
                            w_end = min(w_end, width)
                            slice_h = h_end - i
                            slice_w = w_end - j
                            # also slice dY to match
                            dY_slice = dY[b, out_ch, :slice_h, :slice_w]
                            X_slice = X[b, in_ch, i:h_end, j:w_end]
                        else:
                            X_slice = X[b, in_ch, i:h_end, j:w_end]
                            dY_slice = dY[b, out_ch]

                        # Accumulate gradients for kernel weights
                        dK[out_ch, in_ch, i, j] += np.sum(X_slice * dY_slice) / batch

                         # Accumulate gradients for input
                        dX[b, in_ch, i:h_end, j:w_end] += K[out_ch, in_ch, i, j] * dY_slice
    return dX, dK, db

## Average Pooling Layer (Forward)

Implements average pooling with a specified kernel size and stride.
This uses a vectorized sliding window summation.


In [10]:
def Pooling(X, kernel_size, stride):
    """
    Average pooling operation.

    Parameters:
    - X: Input tensor of shape (batch, channel, height, width)
    - kernel_size: Tuple (kH, kW) specifying pooling window size
    - stride: Stride of the pooling operation

    Returns:
    - Y: Output tensor after pooling
    """

    batch, channel, height, width = X.shape
    output_height = (height - kernel_size[0]) // stride + 1
    output_width  = (width - kernel_size[1]) // stride + 1
    Y = np.zeros(shape=(batch, channel, output_height, output_width))

    """
    First attempt, too slow.
    for b in range(batch):
        for ch in range(channel):
            for h in range(output_height):
                for w in range(output_width):
                    # Calculated this from a piece of paper, helped me understand everything a lot better.
                    Y[b, ch, h, w] = np.mean(X[b, ch, 
                                               (h * stride): (h * stride) + kernel_size[0],
                                               (w * stride): (w * stride) + kernel_size[1]])
    """

    window_sum = np.zeros((batch, channel, output_height, output_width))
    for i in range(kernel_size[0]):
        for j in range(kernel_size[1]):
            window_sum += X[:, :, i:i+output_height*stride:stride, j:j+output_width*stride:stride]
    Y = window_sum / (kernel_size[0] * kernel_size[1])

    return Y

## Average Pooling Backward Pass

Distributes gradients equally over the region covered by each pooling window.


In [11]:
def PoolingBackward(dY, X, kernel_size, stride):
    """
    Backpropagation for average pooling.

    Spreads gradient uniformly over the region that was pooled.

    Parameters:
    - dY: Gradient w.r.t. output, shape (batch, channel, out_height, out_width)
    - X: Input to pooling layer, shape (batch, channel, height, width)
    - kernel_size: Tuple (kH, kW)
    - stride: Pooling stride

    Returns:
    - dX: Gradient w.r.t. input X
    """
    
    batch, channel, height, width = X.shape
    output_height, output_width = dY.shape[2], dY.shape[3]
    kH, kW = kernel_size
    dX = np.zeros_like(X)

    for b in range(batch):
        for ch in range(channel):
            for h in range(output_height):
                for w in range(output_width):
                    dX[b, ch,
                       (h * stride):(h * stride) + kH,
                       (w * stride):(w * stride) + kW] += dY[b, ch, h, w] / (kH * kW)
    return dX

# Weight Initialization

We initialize weights differently depending on the activation function and layer type
to help maintain stable variance and gradients during training:

- **Convolutional layers (tanh activations)**:
  Use LeCun uniform initialization, which samples weights from
  Uniform(-sqrt(3 / fan_in), sqrt(3 / fan_in)).
  This preserves the variance of activations when using tanh.

- **Fully connected layers (sigmoid activations)**:
  Use Xavier initialization scaled for sigmoid, sampling from
  Uniform(-4 * sqrt(6 / (fan_in + fan_out)), 4 * sqrt(6 / (fan_in + fan_out))).
  The scale factor (4) compensates for the maximum slope of sigmoid (~0.25).

- **Biases** are initialized to zeros for all layers.

This initialization reduces the risk of vanishing or exploding activations, which this model suffered from when using default Xavier Initialization on Sigmoid. Gradients vanished really fast.

In [ ]:
def lecun_tanh_init(size, n_in):
    """
    LeCun uniform initialization for weights used with tanh activations.

    Parameters:
    - size : tuple
        Shape of the weight tensor (e.g., (out_channels, in_channels, kernel_h, kernel_w)).
    - n_in : int
        Number of input units (fan_in). For convolution, this is typically
        kernel_height * kernel_width * input_channels.

    Returns:
    - np.ndarray
        Initialized weights sampled from Uniform(-limit, limit),
        where limit = sqrt(3 / n_in).
        This preserves variance for layers using tanh activations.
    """
    limit = np.sqrt(3.0 / n_in)
    return np.random.uniform(-limit, limit, size=size)


def xavier_sigmoid_init(size, n_in, n_out):
    """
    Xavier (Glorot) initialization scaled for sigmoid activations.

    Since sigmoid’s max slope is ~0.25, we use:
    limit = 4 * sqrt(6 / (fan_in + fan_out))

    Parameters:
    - size : tuple
        Shape of the weight matrix (e.g., (output_units, input_units)).
    - n_in : int
        Number of input units (fan_in).
    - n_out : int
        Number of output units (fan_out).

    Returns:
    - np.ndarray
        Initialized weights sampled from Uniform(-limit, limit).
    """
    limit = 4.0 * np.sqrt(6.0 / (n_in + n_out))
    return np.random.uniform(-limit, limit, size=size)

In [ ]:
# LeCun-tanh initialization for each convolutional layer

# C1: 6 filters, 1 input channel, 5x5 kernel
c1_kernel = lecun_tanh_init(size=(6, 1, 5, 5), n_in=25)
c1_bias = np.zeros(6)

# C3: 16 filters, 6 input channels
c3_kernel = lecun_tanh_init(size=(16, 6, 5, 5), n_in=150)
c3_bias = np.zeros(16)

# C5: 120 filters, 16 input channels
c5_kernel = lecun_tanh_init(size=(120, 16, 5, 5), n_in=400)
c5_bias = np.zeros(120)

# Xavier initialization for each fc layer

# F6: fully connected 120 -> 84
f6_weights = xavier_sigmoid_init((84, 120), n_in=120, n_out=84)
f6_bias = np.zeros(84)

# Output layer: fully connected 84 -> 10
out_weights = xavier_sigmoid_init((10, 84), n_in=84, n_out=10)
out_bias = np.zeros(10)

# Training Loop

Trains the network over multiple epochs. Uses SGD to update weights.
Includes explicit forward and backward passes through all layers.
Training on only 4 epochs on 10000 images because it is very slow.

We also shuffle the data each epoch to improve convergence.


In [ ]:
batch_size = 64
epochs = 4
lr = 0.1

for epoch in range(epochs):
    total_loss = 0    # Sum of losses for this epoch
    correct = 0       # Count of correct predictions

    # Shuffle data each epoch
    perm = np.random.permutation(X_train.shape[0])
    X_train = X_train[perm]
    y_train = y_train[perm]
    
    for batch in range(0, X_train.shape[0], batch_size):
        end = min(batch + batch_size, X_train.shape[0])
        X_train_batch = X_train[batch:end]
        y_train_batch = y_train[batch:end]
        batch_len = X_train_batch.shape[0]
    
        # Forward Pass
        c1, X1, c1_kernel, c1_bias = Convolution(X_train_batch,
                                                p=2, 
                                                in_channels=1, 
                                                out_channels=6,
                                                K=c1_kernel,
                                                bias=c1_bias)
        a1 = tanh(c1)
        s2 = Pooling(a1, kernel_size=(2, 2), stride=2)
        c3, _, c3_kernel, c3_bias = Convolution(s2,
                                                p=0,
                                                in_channels=6,
                                                out_channels=16,
                                                K=c3_kernel,
                                                bias=c3_bias
                                                )
        a3 = tanh(c3)
        s4 = Pooling(a3, kernel_size=(2, 2), stride=2)
        c5, _, c5_kernel, c5_bias = Convolution(s4,
                                                p=0,
                                                in_channels=16,
                                                out_channels=120,
                                                K=c5_kernel,
                                                bias=c5_bias
                                                )
        a5 = tanh(c5).reshape(batch_len, 120)
        f6 = np.dot(a5, f6_weights.T) + f6_bias
        a6 = sigmoid(f6)
        output = np.dot(a6, out_weights.T) + out_bias
        yhat = softmax(output)

        # Onehot Encoding Labels
        y_onehot = np.zeros((batch_len, 10))
        y_onehot[np.arange(batch_len), y_train_batch] = 1   # Turn class indices into one-hot vectors

        # Loss and Accuracy
        loss = CrossEntropy(yhat, y_onehot)                 # Average loss over batch
        total_loss += loss * batch_len                      # Not multiplying by batch_len was an error
        preds = np.argmax(yhat, axis=1)                     # Class prediction per sample
        correct += np.sum(preds == y_train_batch)           # Tally correct predictions

        dz_output = yhat - y_onehot
        # Grad for output layer
        dw_out = np.dot(dz_output.T, a6) / batch_len  # shape (10, 84)
        db_out = dz_output.sum(axis=0) / batch_len    # shape (10,)

        # Propagate to hidden layer
        da6 = np.dot(dz_output, out_weights)          # (batch, 84)
        dz6 = da6 * sigmoid_deriv(f6)                 # (batch, 84)

        # Grad for F6 layer
        dw_f6 = np.dot(dz6.T, a5) / batch_len         # shape (84, 120)
        db_f6 = dz6.sum(axis=0) / batch_len           # shape (84,)

        # Propagate to C5 output
        da5 = np.dot(dz6, f6_weights)                 # (batch, 120)
        da5 = da5.reshape(batch_len, 120, 1, 1)       # for compatibility with conv

        # Propagate through C5 layer
        dz5 = da5 * tanh_deriv(c5)
        ds4, dK5, db5 = ConvBackward(dz5, s4, c5_kernel)

        # Propagate through S4
        da3 = PoolingBackward(ds4, a3, (2,2), 2)

        #Propagate through C3
        ds2, dK3, db3 = ConvBackward(da3 * tanh_deriv(c3), s2, c3_kernel)

        #Propagate through S2
        da1 = PoolingBackward(ds2, a1, (2,2), 2)

        #Propagate through C1
        _, dK1, db1 = ConvBackward(da1 * tanh_deriv(c1), X1, c1_kernel)

        """
        def grad_stats(name, grad):
            abs_grad = np.abs(grad)
            print(f"{name:<8} | mean: {abs_grad.mean():.3e}  std: {abs_grad.std():.3e}  min: {abs_grad.min():.3e}  max: {abs_grad.max():.3e}")
        
        grad_stats("dK5", dK5)
        grad_stats("da3", da3)
        grad_stats("dK3", dK3)
        grad_stats("dK1", dK1)
        grad_stats("dw_f6", dw_f6)
        grad_stats("dw_out", dw_out)

        Debugging because accuracy was stagnant at 10% accuracy. Gradients on average
        extremely small. I was using Xavier Initialization but then switched.
        """ 

        out_weights -= lr * dw_out
        out_bias -= lr * db_out
        f6_weights -= lr * dw_f6
        f6_bias -= lr * db_f6
        c5_kernel -= lr * dK5
        c5_bias -= lr * db5
        c3_kernel -= lr * dK3
        c3_bias -= lr * db3
        c1_kernel -= lr * dK1
        c1_bias -= lr * db1

    # Epoch Results
    acc = correct / X_train.shape[0]
    print(f"Epoch {epoch+1} | Loss: {total_loss / X_train.shape[0]:.4f} | Accuracy: {acc:.4f}")

Epoch 1 | Loss: 0.6697 | Accuracy: 0.8205
Epoch 2 | Loss: 0.2234 | Accuracy: 0.9364
Epoch 3 | Loss: 0.1726 | Accuracy: 0.9509
Epoch 4 | Loss: 0.1387 | Accuracy: 0.9627


Saving parameters

In [14]:
np.savez("cnn_params.npz",
         c1_kernel=c1_kernel, c1_bias=c1_bias,
         c3_kernel=c3_kernel, c3_bias=c3_bias,
         c5_kernel=c5_kernel, c5_bias=c5_bias,
         f6_weights=f6_weights, f6_bias=f6_bias,
         out_weights=out_weights, out_bias=out_bias)

In [ ]:
params = np.load("cnn_params.npz")
c1_kernel = params["c1_kernel"]
c1_bias = params["c1_bias"]
c3_kernel = params["c3_kernel"]
c3_bias = params["c3_bias"]
c5_kernel = params["c5_kernel"]
c5_bias = params["c5_bias"]
f6_weights = params["f6_weights"]
f6_bias = params["f6_bias"]
out_weights = params["out_weights"]
out_bias = params["out_bias"]

Running the model on test set

In [12]:
batch_size = 64
correct = 0
total = 0

for batch in range(0, X_test.shape[0], batch_size):
    end = min(batch + batch_size, X_test.shape[0])
    X_test_batch = X_test[batch:end]
    y_test_batch = y_test[batch:end]
    batch_len = X_test_batch.shape[0]

    c1, _, _, _ = Convolution(X_test_batch,
                              p=2, 
                              in_channels=1, 
                              out_channels=6,
                              K=c1_kernel,
                              bias=c1_bias)
    a1 = tanh(c1)
    s2 = Pooling(a1, kernel_size=(2, 2), stride=2)
    c3, _, _, _ = Convolution(s2,
                              p=0,
                              in_channels=6,
                              out_channels=16,
                              K=c3_kernel,
                              bias=c3_bias
                             )
    a3 = tanh(c3)
    s4 = Pooling(a3, kernel_size=(2, 2), stride=2)
    c5, _, _, _ = Convolution(s4,
                              p=0,
                              in_channels=16,
                              out_channels=120,
                              K=c5_kernel,
                              bias=c5_bias
                             )
    a5 = tanh(c5).reshape(batch_len, 120)
    f6 = np.dot(a5, f6_weights.T) + f6_bias
    a6 = sigmoid(f6)
    output = np.dot(a6, out_weights.T) + out_bias
    yhat = softmax(output)

    preds = np.argmax(yhat, axis=1)
    correct += np.sum(preds == y_test_batch)
    total += batch_len

acc = correct / total
print(f"Test Accuracy: {acc:.4f}")

Test Accuracy: 0.9384


# Conclusion

In this project, I successfully implemented the LeNet-5 architecture from scratch in Numpy, including forward and backward passes for convolutions, pooling, and fc layers. It was extremely slow so I trained on only 10,000 images. I then replicated it in PyTorch to train for a longer time. I made two mistakes that took me a long time to debug.

Backpropagation through padding:
Initially, I forgot to account for the padding applied in the first convolutional layer during backpropagation. This led to incorrect gradient computations because the backward pass was slicing over the original unpadded input, mismatching the forward pass, causing a stagnant 10% accuracy (randomly guessing). I fixed this by making my convolution function return the padded input, then making backpropagation use that padded input instead of the default input.

Mismatched activation functions and initialization:
After I fixed the padding error, it was still stagnant at 10% accuracy. I printed out the gradients and saw that all of them were tiny, on verge of vanishing. Even if they didn't vanish, updates would be ridiculously slow. I double checked the architecture and saw that I made a mistake. I used sigmoid activations across all layers, including the convolutional ones, with Xavier initialization. I thought LeNet-5 used only sigmoid as the activation function, but it actually used Tanh for the convolutional layers, and sigmoid for the fc layers. I fixed that and used LeCun as initialization for the convolutional layers, and Xavier initialization with a bit of scaling for the fc layers. This dramatically improved gradient flow and allowed the network to actually learn.

Ultimately, this project not only resulted in a functional LeNet-5 implementation from scratch but also gave me appreciation for engineering optimizations embedded in libraries like PyTorch. They efficiently handle operation and backpropagation details that, when done manually, is extremely slow. Running the PyTorch version on a GPU shortened the training time from 10s of hours to minutes.

This project left me with a much stronger intuition for the data flow, mathematical operations, and design choices inside convolutional neural networks.